## Konkordanser mot bokhylla med emneord

Versjon: **Mars 2019**

Velg `Run all` fra Cell-menyen over. Visning av kode forsvinner, som kan toggles ved å trykke på den grå knappen nedenfor. 

Gå til avsnittet med Konkordanser og fyll inn det som trengs der.

In [11]:
#!curl -s https://raw.githubusercontent.com/yoonsen/modules/master/module_update.py > module_update.py

In [12]:
from module_update import css, code_toggle, printmd

In [13]:
css()

In [14]:
code_toggle()

In [15]:
#from nbtext import get_konk
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline

In [16]:
import pandas as pd
import requests
from IPython.display import HTML

def get_konk(word, params=None, kind='html'):
    if params is None:
        params = {}

    para = params
    para['word']= word

    corpus = 'bok'
    if 'corpus' in para:
        corpus = para['corpus']
    else:
        para['corpus'] = corpus
        
    r = requests.get('https://api.nb.no/ngram/konk', params=para)
    if kind=='html':
        rows = ""
        row_template = ("<tr>"
                        "<td><a href='{urn}?searchText={kw}' target='_'>{urnredux}</a></td>"
                        "<td>{b}</td>"
                        "<td>{w}</td>"
                        "<td style='text-align:left'>{a}</td>"
                        "</tr>\n")
        if corpus == 'bok':
            for x in r.json():
                rows += row_template.format(
                    kw = word,
                    urn=x['urn'],
                    urnredux=','.join([x['author'], x['title'], str(x['year'])]),
                    b=x['before'],
                    w=x['word'],
                    a=x['after'])
        else:
            #print(r.json())
            for x in r.json():
                rows += row_template.format(
                    kw = word,
                    urn=x['urn'],
                    urnredux='-'.join(x['urn'].split('_')[2:6:3]),
                    b=x['before'],
                    w=x['word'],
                    a=x['after'])
        res = "<table>{rows}</table>".format(rows=rows)
        res = HTML(res)
    elif kind == 'json':
        res = r.json()
    else:
        try:
            if corpus == 'bok':
                res = pd.DataFrame(r.json())
                res = res[['urn','author','title','year','before','word','after']]
            else:
                res = pd.DataFrame(r.json())
                res = res[['urn','before','word','after']]
            
        except:
            res= pd.DataFrame()
        #r = r.style.set_properties(subset=['after'],**{'text-align':'left'})
    return res


## Konkordanser
Finn konkordanser. Fyll in søkeord og emneord og klikk knappen `Hent konkordanser!`, eller bruk returtasten. Repeter klikk og tastetrykk for å få flere resultat.

Både emneord og søkeord er følsom for stor forbokstav. Med liten forbokstav er emneordene fritt laget, mens de kapitaliserte tilhører en tesaurus som for eksempel Humord.

In [17]:
res = dict()

In [18]:
def konk(søkeord='', emneord='', forfatter ='', tittel ='', 
         periode =widgets.IntRangeSlider(min=1720, max= 2010, value=[1950, 2000]), 
         før = (0,12), etter = (0,12),  grense = (0,10)):
    res[søkeord] = get_konk(søkeord, {'yearfrom': periode[0], 'author': '%'.join(forfatter.split()) + "%", 'subject':emneord, 'title': "%" + '%'.join(tittel.split()) + "%", 'yearto':periode[1], 'corpus':'bok', 'before':før, 'after':etter, 'size':grense})
    return res[søkeord]

im = interact_manual(konk);
im.widget.children[8].description = 'Hent konkordanser!'

interactive(children=(Text(value='', description='søkeord'), Text(value='', description='emneord'), Text(value…